In [46]:
import numpy as np
import scipy.sparse as sp

# Construct A1=\partial_{x}^{2}+\partial_{y}^{2}


def create_laplacian_matrix(n, h):
    """
    Constructs the matrix A for the discrete Laplacian operator with periodic boundary conditions.

    Parameters:
    n (int): The number of grid points along one dimension (assuming an n x n grid).
    h (float): The grid spacing.

    Returns:
    scipy.sparse.csr_matrix: The Laplacian matrix A in sparse format.
    """
    N = n * n  # Total number of points in the grid
    main_diag = -4 * np.ones(N)  # Main diagonal (central point effect)
    side_diag = np.ones(N - 1)   # Side diagonals for left-right neighbors 
    up_down_diag = np.ones(N - n) # Diagonals for top-bottom neighbors

    # Set up diagonals with periodic boundary conditions
    diagonals = [main_diag, side_diag, side_diag, up_down_diag, up_down_diag]
    offsets = [0, 1, -1, n, -n]

    # Create sparse matrix with diagonals
    A = sp.diags(diagonals, offsets, shape=(N, N), format='csr')

    # The following for loop is dealing with the Periodic boundary conditions
    for i in range(n):
        '''
        For each loop, we are dealing with the boundary condition of the i-th row and i-th column.
        '''
        A[i * n, i * n + n - 1] = 1  # Left edge connected to right edge
        A[i * n + n - 1, i * n] = 1  # Right edge connected to left edge
        A[i, i + (n - 1) * n] = 1    # Top edge connected to bottom edge
        A[i + (n - 1) * n, i] = 1    # Bottom edge connected to top edge

        if i < n - 1:
            A[(i + 1) * n - 1, (i + 1) * n] = 0  # No right neighbor for the last column
            A[(i + 1) * n, (i + 1) * n - 1] = 0  # No left neighbor for the first column

    A = A / h**2  # Multiply by 1/h^2

    return A

n = 8  # Grid size
h = 20/n
A = create_laplacian_matrix(n, h)
A1 = A.toarray() # Convert to dense array for display (optional)
print("Matrix A is")
print(A1)

Matrix A is
[[-0.64  0.16  0.   ...  0.    0.    0.  ]
 [ 0.16 -0.64  0.16 ...  0.    0.    0.  ]
 [ 0.    0.16 -0.64 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -0.64  0.16  0.  ]
 [ 0.    0.    0.   ...  0.16 -0.64  0.16]
 [ 0.    0.    0.   ...  0.    0.16 -0.64]]


In [47]:
# Construct A2 = Matrix B =\partial_{x}

def create_y_derivative_matrix(n, h):
    """
    Constructs the matrix B for the discrete x-derivative operator with periodic boundary conditions.

    Parameters:
    n (int): The number of grid points along one dimension (assuming an n x n grid).
    h (float): The grid spacing.

    Returns:
    scipy.sparse.csr_matrix: The x-derivative matrix B in sparse format.
    """
    N = n * n  # Total number of points in the grid
    main_diag = np.zeros(N)  # Main diagonal (central point effect)
    up_down_diag = np.ones(N - n)  # Diagonals for top-bottom neighbors

    # Set up diagonals with periodic boundary conditions
    diagonals = [main_diag, up_down_diag, -up_down_diag]
    offsets = [0, n, -n]

    # Create sparse matrix with diagonals
    B = sp.diags(diagonals, offsets, shape=(N, N), format='csr')

    # The following for loop is dealing with the Periodic boundary conditions
    for i in range(n):
        '''
        For each loop, we are dealing with the boundary condition of the i-th row and i-th column.
        '''
        B[i, i + (n - 1) * n] = -1  # Top edge connected to bottom edge
        B[i + (n - 1) * n, i] = 1  # Bottom edge connected to top edge

    B = B  / (2 * h)  # Multiply by 1/(2h)

    return B

n = 8  # Grid size
h = 20/8  # Assuming a grid over [-10, 10]
# h=0.5 # Just for testing
B = create_y_derivative_matrix(n, h)
A2 = B.toarray() # Convert to dense array for display (optional)
print("Matrix B is")
print(A2)

Matrix B is
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [48]:
# Construct A3 = Matrix C =\partial_{y}

def create_x_derivative_matrix(n, h):
    """
    Constructs the matrix C for the discrete y-derivative operator with periodic boundary conditions.

    Parameters:
    n (int): The number of grid points along one dimension (assuming an n x n grid).
    h (float): The grid spacing.

    Returns:
    scipy.sparse.csr_matrix: The y-derivative matrix C in sparse format.
    """
    N = n * n  # Total number of points in the grid
    main_diag = np.zeros(N)  # Main diagonal (central point effect)
    side_diag = np.ones(N - 1)  # Side diagonals for left-right neighbors

    # Set up diagonals with periodic boundary conditions
    diagonals = [main_diag, side_diag, -side_diag]
    offsets = [0, 1, -1]

    # Create sparse matrix with diagonals
    C = sp.diags(diagonals, offsets, shape=(N, N), format='csr')

    # The following for loop is dealing with the Periodic boundary conditions
    for i in range(n):
        '''
        For each loop, we are dealing with the boundary condition of the i-th row and i-th column.
        '''
        C[i * n, i * n + n - 1] = -1  # Left edge connected to right edge
        C[i * n + n - 1, i * n] = 1  # Right edge connected to left edge

        if i < n - 1:
            C[(i + 1) * n - 1, (i + 1) * n] = 0 # No right neighbor for the last column
            C[(i + 1) * n, (i + 1) * n - 1] = 0 # No left neighbor for the first column

    C = C / (2 * h)  # Multiply by 1/(2h)

    return C

n = 8  # Grid size
h = 20/n  # Assuming a grid over [-10, 10]
# h=0.5 # Just for testing
C = create_x_derivative_matrix(n, h)
A3 = C.toarray() # Convert to dense array for display (optional)
print("Matrix C is:")
print(A3)


Matrix C is:
[[ 0.   0.2  0.  ...  0.   0.   0. ]
 [-0.2  0.   0.2 ...  0.   0.   0. ]
 [ 0.  -0.2  0.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.   0.2  0. ]
 [ 0.   0.   0.  ... -0.2  0.   0.2]
 [ 0.   0.   0.  ...  0.  -0.2  0. ]]
